In [1]:
# !pip install demoji

In [2]:
import datetime
from glob import glob
import time
import os
import re
import pandas as pd
import demoji
demoji.download_codes()

def print_full(x):
    pd.set_option('display.max_rows', len(x))
    pd.set_option('display.max_columns', None)
    pd.set_option('display.width', 2000)
    pd.set_option('display.float_format', '{:20,.2f}'.format)
    pd.set_option('display.max_colwidth', -1)
    print(x)
    pd.reset_option('display.max_rows')
    pd.reset_option('display.max_columns')
    pd.reset_option('display.width')
    pd.reset_option('display.float_format')
    pd.reset_option('display.max_colwidth')

pd.options.display.max_rows
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', -1)

chunksize = 10 ** 6
date_time = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
print("\ndate_time = {}".format(date_time))

out_folder = "out_{}".format(date_time)

if not os.path.exists(out_folder):
    os.mkdir(out_folder)
    print("Directory " , out_folder ,  " Created ")
else:    
    print("Directory " , out_folder ,  " already exists")

# df = pd.read_csv("data/out_35.csv")
# df = pd.read_csv("data/subset_1.csv", encoding='utf8', dtype=str)
# print(df.count())
# df.head()


... OK (Got response in 0.86 seconds)
Writing emoji data to C:\Users\USER\.demoji/codes.json ...
... OK

date_time = 20200414_105818
Directory  out_20200414_105818  Created 


C:\Users\USER\anaconda3\lib\site-packages\ipykernel_launcher.py:27: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.


In [3]:
def filterDuplicate(df):
    import time
    time_start = time.time()
    print("  -> filterDuplicate()", end='')
    
    remove = df.duplicated(keep='first')
    result = df[~remove]
    
#     print("Removed: {} \n".format(df[remove].head()))
    
    processed = len(df.index)
    skipped = processed - len(result.index)
    skipped_percentage = skipped / processed * 100
    
    time = time.time() - time_start
    print(" - Processed: {:,} | Skipped: {:,} ({:.2f}%) | Time: {:,.3f} sec".format(processed, skipped, skipped_percentage, time))
    
    return result


In [4]:
def filterDuplicateOriginalTweetId(df):
    import time
    time_start = time.time()
    print("  -> filterDuplicateOriginalTweetId()", end='')

    file_name = "{}/df_original_tweet_ids.csv".format(out_folder)
    if not os.path.exists(file_name):
        original_tweet_ids = pd.DataFrame(columns=['original_tweet_id'], dtype=str)
        original_tweet_ids.to_csv(file_name, index=False, encoding='utf-8')
    
    remove = ~df.original_tweet_id.notnull()
    remove = remove | df.duplicated(subset=['original_tweet_id'], keep='first')
    
    for original_tweet_ids in pd.read_csv(file_name, encoding='utf-8', dtype=str, chunksize=chunksize):
        remove = remove | df.original_tweet_id.isin(original_tweet_ids.original_tweet_id)
        
    result = df[~remove]
    
    original_tweet_ids = pd.DataFrame(columns=['original_tweet_id'], dtype=str)
    original_tweet_ids = pd.concat([original_tweet_ids, pd.DataFrame({'original_tweet_id': result.original_tweet_id})])
    original_tweet_ids.to_csv(file_name, mode='a', header=False, index=False, encoding='utf-8')
    
    remove = remove & (df.is_retweet.isin(['True']) | df.is_quote.isin(['True']))
    result = df[~remove]
    
#     print("Removed: {} \n".format(df[remove].head()))
    
    processed = len(df.index)
    skipped = processed - len(result.index)
    skipped_percentage = skipped / processed * 100
    
    time = time.time() - time_start
    print(" - Processed: {:,} | Skipped: {:,} ({:.2f}%) | Time: {:,.3f} sec".format(processed, skipped, skipped_percentage, time))
    
    return result

# subsets = [file for file in glob('data/subset_*.csv'.format())]
# subsets = sorted(subsets)
# for file_name in subsets:
#     print("\nProcessing '{}'".format(file_name))
#     df = pd.read_csv(file_name, encoding='utf8', dtype=str)
#     m = df[df['original_tweet_id']=='1236850730163560448']
#     print(m.head(100))
#     print()


In [5]:
def filterUrl(df):
    import time
    time_start = time.time()
    print("  -> filterUrl()", end='')
    
    url_regex = r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\), ]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
    df['text'] = df['text'].str.replace(url_regex, '').astype(str)
    df['quoted_text'] = df['quoted_text'].str.replace(url_regex, '').astype(str)
    
    processed = len(df.index)
    time = time.time() - time_start
    print(" - Processed: {:,} | Time: {:,.3f} sec".format(processed, time))
    
    return df


In [6]:
def checkNonEngAndEmoji(row):
    text = re.sub(r'…', '', str(row['text']))
    quoted_text = re.sub(r'…', '', str(row['quoted_text']))
    
    text_emoji = demoji.replace(text, '')
    if len(text_emoji) != len(text):
        row['has_emoji'] = 'True'
    quoted_text_emoji = demoji.replace(quoted_text, '')
    if len(quoted_text_emoji) != len(quoted_text):
        row['has_emoji'] = 'True'
    
    try:
        text_emoji.encode(encoding='utf-8').decode('ascii')
        quoted_text_emoji.encode(encoding='utf-8').decode('ascii')
    except UnicodeDecodeError:
        row['has_non_eng'] = 'True'
        
    return row


def filterNonEngAndEmoji(df):
    import time
    time_start = time.time()
    print("  -> filterNonEngAndEmoji()", end='')
    
    df['has_emoji'] = 'False'
    df['has_non_eng'] = 'False'
    
    df = df.apply(checkNonEngAndEmoji, axis=1)
    
    remove = df.has_non_eng.isin(['True'])
    result = df[~remove]
    
    processed = len(df.index)
    skipped = processed - len(result.index)
    skipped_percentage = skipped / processed * 100
    time = time.time() - time_start
    print(" - Processed: {:,} | Non-Eng Skipped: {:,} ({:.2f}%) | Time: {:,.3f} sec".format(processed, skipped, skipped_percentage, time))
    
    return result


In [7]:
subsets = [file for file in glob('data/tweet_files/out_*.csv'.format())]
subsets = sorted(subsets)
print("List of csv: {}\n".format(subsets))

for file_name in subsets:
    print("\nProcessing '{}'".format(file_name))
    df = pd.read_csv(file_name, encoding='utf8', dtype=str,error_bad_lines=False)
    df = filterDuplicate(df)
    df = filterDuplicateOriginalTweetId(df)
    df = filterUrl(df)
    df = filterNonEngAndEmoji(df)
    
    out_file_name = "{}/combined.csv".format(out_folder)
    if not os.path.exists(out_file_name):
        df.to_csv(out_file_name, mode='w', header=True, index=False, encoding='utf-8')
    else:
        df.to_csv(out_file_name, mode='a', header=False, index=False, encoding='utf-8')


List of csv: ['data/tweet_files\\out_37.csv', 'data/tweet_files\\out_38.csv', 'data/tweet_files\\out_39.csv', 'data/tweet_files\\out_40.csv', 'data/tweet_files\\out_41.csv', 'data/tweet_files\\out_42.csv', 'data/tweet_files\\out_43.csv', 'data/tweet_files\\out_44.csv', 'data/tweet_files\\out_45.csv', 'data/tweet_files\\out_46.csv', 'data/tweet_files\\out_47.csv', 'data/tweet_files\\out_48.csv', 'data/tweet_files\\out_49.csv', 'data/tweet_files\\out_50.csv', 'data/tweet_files\\out_51.csv', 'data/tweet_files\\out_52.csv', 'data/tweet_files\\out_53.csv', 'data/tweet_files\\out_54.csv', 'data/tweet_files\\out_55.csv', 'data/tweet_files\\out_56.csv', 'data/tweet_files\\out_57.csv', 'data/tweet_files\\out_58.csv', 'data/tweet_files\\out_59.csv', 'data/tweet_files\\out_60.csv', 'data/tweet_files\\out_61.csv', 'data/tweet_files\\out_62.csv', 'data/tweet_files\\out_63.csv', 'data/tweet_files\\out_64.csv', 'data/tweet_files\\out_65.csv', 'data/tweet_files\\out_66.csv', 'data/tweet_files\\out_67.

b'Skipping line 521165: expected 16 fields, saw 17\n'
b'Skipping line 806035: expected 16 fields, saw 17\n'


  -> filterDuplicate() - Processed: 999,981 | Skipped: 197 (0.02%) | Time: 2.586 sec
  -> filterDuplicateOriginalTweetId() - Processed: 999,784 | Skipped: 769,799 (77.00%) | Time: 0.738 sec
  -> filterUrl() - Processed: 229,985 | Time: 0.594 sec
  -> filterNonEngAndEmoji() - Processed: 229,985 | Non-Eng Skipped: 100,547 (43.72%) | Time: 169.204 sec

Processing 'data/tweet_files\out_39.csv'


b'Skipping line 459654: expected 16 fields, saw 17\n'


  -> filterDuplicate() - Processed: 999,990 | Skipped: 139 (0.01%) | Time: 2.805 sec
  -> filterDuplicateOriginalTweetId() - Processed: 999,851 | Skipped: 827,807 (82.79%) | Time: 0.782 sec
  -> filterUrl() - Processed: 172,044 | Time: 0.471 sec
  -> filterNonEngAndEmoji() - Processed: 172,044 | Non-Eng Skipped: 74,150 (43.10%) | Time: 126.338 sec

Processing 'data/tweet_files\out_40.csv'


...

  -> filterDuplicate() - Processed: 999,953 | Skipped: 131 (0.01%) | Time: 2.642 sec
  -> filterDuplicateOriginalTweetId() - Processed: 999,822 | Skipped: 739,193 (73.93%) | Time: 5.327 sec
  -> filterUrl() - Processed: 260,629 | Time: 0.726 sec
  -> filterNonEngAndEmoji() - Processed: 260,629 | Non-Eng Skipped: 109,692 (42.09%) | Time: 195.329 sec

Processing 'data/tweet_files\out_87.csv'
  -> filterDuplicate() - Processed: 1,014,419 | Skipped: 14,899 (1.47%) | Time: 2.687 sec
  -> filterDuplicateOriginalTweetId() - Processed: 999,520 | Skipped: 727,115 (72.75%) | Time: 5.545 sec
  -> filterUrl() - Processed: 272,405 | Time: 0.747 sec
  -> filterNonEngAndEmoji() - Processed: 272,405 | Non-Eng Skipped: 116,742 (42.86%) | Time: 207.198 sec

Processing 'data/tweet_files\out_88.csv'
  -> filterDuplicate() - Processed: 696,680 | Skipped: 138 (0.02%) | Time: 1.682 sec
  -> filterDuplicateOriginalTweetId() - Processed: 696,542 | Skipped: 509,356 (73.13%) | Time: 5.266 sec
  -> filterUrl() - 

b'Skipping line 270233: expected 16 fields, saw 17\nSkipping line 284868: expected 16 fields, saw 17\n'
b'Skipping line 705253: expected 16 fields, saw 17\n'


  -> filterDuplicate() - Processed: 894,682 | Skipped: 192 (0.02%) | Time: 2.428 sec
  -> filterDuplicateOriginalTweetId() - Processed: 894,490 | Skipped: 651,761 (72.86%) | Time: 5.666 sec
  -> filterUrl() - Processed: 242,729 | Time: 0.662 sec
  -> filterNonEngAndEmoji() - Processed: 242,729 | Non-Eng Skipped: 107,367 (44.23%) | Time: 185.724 sec

Processing 'data/tweet_files\out_92.csv'
  -> filterDuplicate() - Processed: 1,418 | Skipped: 0 (0.00%) | Time: 0.006 sec
  -> filterDuplicateOriginalTweetId() - Processed: 1,418 | Skipped: 858 (60.51%) | Time: 4.686 sec
  -> filterUrl() - Processed: 560 | Time: 0.002 sec
  -> filterNonEngAndEmoji() - Processed: 560 | Non-Eng Skipped: 250 (44.64%) | Time: 0.438 sec

Processing 'data/tweet_files\out_93.csv'
  -> filterDuplicate() - Processed: 999,975 | Skipped: 168 (0.02%) | Time: 2.569 sec
  -> filterDuplicateOriginalTweetId() - Processed: 999,807 | Skipped: 707,459 (70.76%) | Time: 5.879 sec
  -> filterUrl() - Processed: 292,348 | Time: 0.

b'Skipping line 320910: expected 16 fields, saw 17\n'


  -> filterDuplicate() - Processed: 349,367 | Skipped: 59 (0.02%) | Time: 0.816 sec
  -> filterDuplicateOriginalTweetId() - Processed: 349,308 | Skipped: 256,004 (73.29%) | Time: 5.537 sec
  -> filterUrl() - Processed: 93,304 | Time: 0.264 sec
  -> filterNonEngAndEmoji() - Processed: 93,304 | Non-Eng Skipped: 39,691 (42.54%) | Time: 71.283 sec

Processing 'data/tweet_files\out_96.csv'
  -> filterDuplicate() - Processed: 108,102 | Skipped: 43 (0.04%) | Time: 0.249 sec
  -> filterDuplicateOriginalTweetId() - Processed: 108,059 | Skipped: 72,697 (67.28%) | Time: 5.105 sec
  -> filterUrl() - Processed: 35,362 | Time: 0.102 sec
  -> filterNonEngAndEmoji() - Processed: 35,362 | Non-Eng Skipped: 15,342 (43.39%) | Time: 27.148 sec

Processing 'data/tweet_files\out_97.csv'
  -> filterDuplicate() - Processed: 1,002,877 | Skipped: 3,232 (0.32%) | Time: 2.701 sec
  -> filterDuplicateOriginalTweetId() - Processed: 999,645 | Skipped: 712,799 (71.31%) | Time: 6.316 sec
  -> filterUrl() - Processed: 2

In [8]:
out_file_name = "{}/combined.csv".format(out_folder)
combined = pd.read_csv(out_file_name, encoding='utf8', dtype=str)

dup = combined[combined.duplicated(keep='first')]
print(dup.count())
dup.head()


Unnamed: 0           3
date                 0
user                 0
is_retweet           0
is_quote             0
text                 0
quoted_text          0
lat                  0
long                 0
hts                  0
mentions             0
tweet_id             0
likes                0
retweets             0
replies              0
quote_count          0
original_tweet_id    0
has_emoji            3
has_non_eng          3
dtype: int64


,Unnamed: 0,date,user,is_retweet,is_quote,text,quoted_text,lat,long,hts,mentions,tweet_id,likes,retweets,replies,quote_count,original_tweet_id,has_emoji,has_non_eng
432264,As a result of the outbreak of new coronavirus CN chaelisa fans did something,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False
432265,A total of 1926.52 yuan was crowdfunded for this time,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False
432266,Donated two disinfection vehicles to Hankou hospital Wuhan totaling 504.85 yuan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False
